# epic 4(sh)

Als een key user kan ik voor een contact met weinig transacties een lookalike met veel transacties identificeren. Ik kan ook een clustering maken van contactpersonen die qua jobinhoud, type bedrijf, voorkeuren en (verwacht) gedrag

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [2]:
ENV_URL = os.path.join(os.getcwd(), '../.env')
load_dotenv(ENV_URL)

DWH_NAME = os.environ.get('DWH_NAME')
SERVER_NAME = os.environ.get('SERVER_NAME')
DB_USER = os.environ.get('DB_USER')
DB_PASSWORD = os.environ.get('DB_PASSWORD')

URL = f'mssql+pymssql://{DB_USER}:{DB_PASSWORD}@{SERVER_NAME}/{DWH_NAME}'
URL_LOCAL = f'mssql+pyodbc://{SERVER_NAME}/{DWH_NAME}?trusted_connection=yes&driver=ODBC+Driver+17 for SQL Server'

engine = create_engine(URL_LOCAL)
conn = engine.connect()

In [3]:
def create_query(table_name, columns, condition=None):

    query = f"SELECT "

    for i, column in enumerate(columns):
        if i == 0:
            query += f"[{column}]"
        else:
            query += f", [{column}]"
    
    query += f" FROM [{DWH_NAME}].[dbo].[{table_name}]"
    
    if condition:
        query += f" WHERE {condition}"

    return query

### 1 Alles van account selecteren

In [4]:
acc_cols = ['accountID', 'plaats','subregio','ondernemingsaard','ondernemingstype','activiteitNaam']
# account conditie
acc_condition = "provincie = 'Oost-Vlaanderen'"
# create query
acc_query = create_query('DimAccount', acc_cols, acc_condition)
# read sql
df_account = pd.read_sql(acc_query, conn)
df_account.shape

(4124, 6)

### 2 Mergen van contact en account

In [5]:
contact_cols = ['contactID', 'accountID', 'functietitel','functieNaam']

contact_query = create_query('DimContact', contact_cols)
df_contact = pd.read_sql(contact_query, conn)

df_contact['functietitel'] = df_contact['functietitel'].str.lower()

df_contact['functieNaam'] = df_contact['functieNaam'].str.lower()
df_contact.shape

(194192, 4)

In [6]:
accounts_merged = pd.merge(df_contact, df_account, on='accountID', how='inner')
accounts_merged.shape

(61885, 9)

### 3 Mergen van account en afspraak

In [7]:
afspraak_cols = ['accountID', 'keyphrases']

afspraak_query = create_query('DimAfspraak', afspraak_cols)

df_afspraak = pd.read_sql(afspraak_query, conn)
df_afspraak.shape

(7167, 2)

In [8]:
acc_con_afs = pd.merge(accounts_merged, df_afspraak, on='accountID', how='inner')
acc_con_afs.shape

(63673, 10)

### 4 Mergen van Campagne en account

In [9]:
campagne_cols = ['campagneID','campagneType','campagneNaam','campagneSoort']

campagne_query = create_query('DimCampagne', campagne_cols)

df_campagne = pd.read_sql(campagne_query, conn)
df_campagne.shape

(4101, 4)

In [10]:
factInschrijving_cols = ['campagneID','contactID']

factInschrijving_query = create_query('FactInschrijving', factInschrijving_cols)

df_factInschrijving = pd.read_sql(factInschrijving_query, conn)
df_factInschrijving.shape

(78790, 2)

In [11]:
camp_fact = pd.merge(df_campagne, df_factInschrijving, on='campagneID', how='inner')
camp_fact.shape

(78790, 5)

In [12]:
df = pd.merge(acc_con_afs, camp_fact, on='contactID', how='inner')
df = df.drop_duplicates(subset=['contactID','campagneID'], keep='first')
df.shape

(12963, 14)

## Data Cleaning

In [13]:
# combine all the data 
df3 = df.copy()
df2 = df
df2['data'] =df[df.columns[1:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
print(df['data'].head())

0    41037B31-0969-E111-B43A-00505680000A,teamleide...
2    41037B31-0969-E111-B43A-00505680000A,student,m...
5    41037B31-0969-E111-B43A-00505680000A,marketing...
6    41037B31-0969-E111-B43A-00505680000A,community...
7    41037B31-0969-E111-B43A-00505680000A,community...
Name: data, dtype: object


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized= vectorizer.fit_transform(df2['data'])

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

#duurt 2m38s
similarities = cosine_similarity(vectorized)

In [16]:
print(similarities)


[[1.         0.77688604 0.7901671  ... 0.09248297 0.09248297 0.10489908]
 [0.77688604 1.         0.79672462 ... 0.07770873 0.07770873 0.09065968]
 [0.7901671  0.79672462 1.         ... 0.11065206 0.11065206 0.1383143 ]
 ...
 [0.09248297 0.07770873 0.11065206 ... 1.         0.88405797 0.859496  ]
 [0.09248297 0.07770873 0.11065206 ... 0.88405797 1.         0.859496  ]
 [0.10489908 0.09065968 0.1383143  ... 0.859496   0.859496   1.        ]]


In [17]:
df = pd.DataFrame(similarities,columns=df['contactID'],index=df['contactID']).reset_index()
df.head()

contactID,contactID,0542DA63-2C64-ED11-9561-6045BD895B5A,09E85092-AF88-EC11-93B0-6045BD91D362,153B9FE0-68BA-E811-80F4-001DD8B72B62,1FE13719-73A4-EC11-983F-00224884C0D3,1FE13719-73A4-EC11-983F-00224884C0D3,1FE13719-73A4-EC11-983F-00224884C0D3,225548F9-8EDA-E711-80EE-001DD8B72B61,225548F9-8EDA-E711-80EE-001DD8B72B61,225548F9-8EDA-E711-80EE-001DD8B72B61,...,E8B5ABFC-C8E6-E611-80E5-001DD8B72B61,E8B5ABFC-C8E6-E611-80E5-001DD8B72B61,E8B5ABFC-C8E6-E611-80E5-001DD8B72B61,F39FA0B4-5736-E711-80E6-001DD8B72B61,F39FA0B4-5736-E711-80E6-001DD8B72B61,FC89CE9B-17CA-E711-80EC-001DD8B72B62,FC89CE9B-17CA-E711-80EC-001DD8B72B62,FC89CE9B-17CA-E711-80EC-001DD8B72B62,FC89CE9B-17CA-E711-80EC-001DD8B72B62,FC89CE9B-17CA-E711-80EC-001DD8B72B62
0,0542DA63-2C64-ED11-9561-6045BD895B5A,1.000000,0.776886,0.790167,0.903343,0.731732,0.742032,0.790167,0.733435,0.803279,...,0.162964,0.174236,0.158285,0.166325,0.151024,0.120714,0.109495,0.092483,0.092483,0.104899
1,09E85092-AF88-EC11-93B0-6045BD91D362,0.776886,1.000000,0.796725,0.764455,0.770595,0.731925,0.796725,0.773137,0.809945,...,0.127802,0.122977,0.124132,0.149071,0.133243,0.106502,0.094632,0.077709,0.077709,0.090660
2,153B9FE0-68BA-E811-80F4-001DD8B72B62,0.790167,0.796725,1.000000,0.777524,0.750417,0.744438,0.810345,0.769259,0.823791,...,0.185695,0.160817,0.162327,0.227429,0.174241,0.154746,0.128333,0.110652,0.110652,0.138314
3,1FE13719-73A4-EC11-983F-00224884C0D3,0.903343,0.764455,0.777524,1.000000,0.832027,0.841270,0.777524,0.721699,0.790425,...,0.160357,0.171448,0.155752,0.181848,0.148608,0.118783,0.107743,0.091003,0.091003,0.117966
4,1FE13719-73A4-EC11-983F-00224884C0D3,0.731732,0.770595,0.750417,0.832027,1.000000,0.832027,0.750417,0.727496,0.764253,...,0.143684,0.172825,0.174448,0.183309,0.149801,0.089803,0.108609,0.091734,0.091734,0.118914


In [19]:
input_person_id = '0542DA63-2C64-ED11-9561-6045BD895B5A'
recommendations = pd.DataFrame(df.nlargest(11,input_person_id)['contactID'])
recommendations = recommendations[recommendations['contactID']!=input_person_id]
print(recommendations)

                               contactID
9   236D00C5-A765-ED11-9561-6045BD8956C9
21  39DF7E64-2C66-ED11-9561-6045BD895BFB
72  EBBF1717-7556-ED11-BBA2-6045BD895BFB
49  945F58FA-9A6C-ED11-9561-6045BD895B5A
3   1FE13719-73A4-EC11-983F-00224884C0D3
60  BA8CB7AB-A2C9-E911-8105-001DD8B72B62
17  29B0EB66-82DA-E711-80EE-001DD8B72B61
42  8F165595-90DA-E711-80EE-001DD8B72B61
69  DE9E5F4B-8349-E811-80F0-001DD8B72B62
65  DE9E5F4B-8349-E811-80F0-001DD8B72B62


## test

In [20]:
df3[(df3['contactID']=='0542DA63-2C64-ED11-9561-6045BD895B5A')].head()

,contactID,accountID,functietitel,functieNaam,plaats,subregio,ondernemingsaard,ondernemingstype,activiteitNaam,keyphrases,campagneID,campagneType,campagneNaam,campagneSoort
0,0542DA63-2C64-ED11-9561-6045BD895B5A,41037B31-0969-E111-B43A-00505680000A,"teamleider graduaatsopleidingen enw bmg, artev...",medewerker,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,"workshop, plantages, student, pakket, onli...",317CD023-2B1E-ED11-B83D-000D3AAD783A,Netwerkevenement,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,Offline


In [30]:
df3[(df3['contactID']=='00008922-92DA-E711-80EE-001DD8B72B61')].head()

,contactID,accountID,functietitel,plaats,subregio,ondernemingsaard,ondernemingstype,activiteitNaam,keyphrases,campagneID,campagneType,campagneNaam,campagneSoort
0,00008922-92DA-E711-80EE-001DD8B72B61,41037B31-0969-E111-B43A-00505680000A,unknown,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,"profiel, tak, social, china, restaurant, ...",NaN,NaN,NaN,NaN


In [31]:
df.head().columns

Index(['contactID', '00008922-92DA-E711-80EE-001DD8B72B61',
       '01C7C34F-90DA-E711-80EE-001DD8B72B61',
       '0217E600-1375-E211-A85C-005056B06EC4',
       '0542DA63-2C64-ED11-9561-6045BD895B5A',
       '09E85092-AF88-EC11-93B0-6045BD91D362',
       '0B39B000-708D-EC11-B400-000D3A24B6E5',
       '126C6BF1-6A9A-E511-A092-005056B06EB4',
       '13AFFF54-DA36-E711-80E6-001DD8B72B61',
       '143B2AE3-1B6F-E111-B43A-00505680000A',
       ...
       'CDC9B99B-61DA-EC11-BB3D-6045BD8B2E69',
       '03ECA1DC-3267-EE11-9AE7-6045BD895B5A',
       '47233332-2B05-EE11-8F6E-6045BD895420',
       'AA8614A8-EA67-EE11-9AE7-6045BD9745FB',
       '700DB6C4-25CD-E811-80F5-001DD8B72B62',
       'F43AC180-E2C9-EC11-A7B5-00224881D030',
       '7713E350-0556-EE11-BE6E-6045BD895420',
       '1F0BD26E-6C66-EE11-9AE7-6045BD895B5A',
       'FCDBE8E9-3868-EE11-9AE7-000D3A4AB78E',
       '1C2A0BA5-899A-E511-A092-005056B06EB4'],
      dtype='object', name='contactID', length=26349)

In [21]:
df3

,contactID,accountID,functietitel,functieNaam,plaats,subregio,ondernemingsaard,ondernemingstype,activiteitNaam,keyphrases,campagneID,campagneType,campagneNaam,campagneSoort
0,0542DA63-2C64-ED11-9561-6045BD895B5A,41037B31-0969-E111-B43A-00505680000A,"teamleider graduaatsopleidingen enw bmg, artev...",medewerker,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,"workshop, plantages, student, pakket, onli...",317CD023-2B1E-ED11-B83D-000D3AAD783A,Netwerkevenement,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,Offline
2,09E85092-AF88-EC11-93B0-6045BD91D362,41037B31-0969-E111-B43A-00505680000A,student,medewerker,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,"workshop, plantages, student, pakket, onli...",C6D72260-E451-EC11-8C62-000D3ABFCF4A,Netwerkevenement,OV-NW-VokaUpdate-Big Refresh-Voorjaar 2022,Online
5,153B9FE0-68BA-E811-80F4-001DD8B72B62,41037B31-0969-E111-B43A-00505680000A,marketingverantwoordelijke,medewerker marketing,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,"workshop, plantages, student, pakket, onli...",7835D8C0-F488-E811-80F3-001DD8B72B61,Netwerkevenement,OV-NW-Verderkijkers 2018-Think Customer,Offline
6,1FE13719-73A4-EC11-983F-00224884C0D3,41037B31-0969-E111-B43A-00505680000A,community manager & coördinator postgraduaat o...,medewerker,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,"workshop, plantages, student, pakket, onli...",317CD023-2B1E-ED11-B83D-000D3AAD783A,Netwerkevenement,OV-NW-Nieuwjaarsreceptie regio Oost-Vlaanderen,Offline
7,1FE13719-73A4-EC11-983F-00224884C0D3,41037B31-0969-E111-B43A-00505680000A,community manager & coördinator postgraduaat o...,medewerker,Gent,Gent,Diensten,Onderwijs,Overige industrie & diensten,"workshop, plantages, student, pakket, onli...",4C8FF159-145C-EC11-8F8F-000D3A2BCF4B,Infosessie,OV-JO Breakfastclub april 2022,Offline
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72039,FC89CE9B-17CA-E711-80EC-001DD8B72B62,4AFDE787-17CA-E711-80EC-001DD8B72B62,unknown,"contact lidmaatschap, bedrijfsleider",ZOTTEGEM,Aalst,Productie & Diensten,Bedrijf,Bouw,"workshop, tijdsinvester, creatief, aankled,...",24D8FBE3-7BB6-E811-80F4-001DD8B72B62,Netwerkevenement,OV-NW-Voka Ambassadeur - Verkiezing 2018,Offline
72040,FC89CE9B-17CA-E711-80EC-001DD8B72B62,4AFDE787-17CA-E711-80EC-001DD8B72B62,unknown,"contact lidmaatschap, bedrijfsleider",ZOTTEGEM,Aalst,Productie & Diensten,Bedrijf,Bouw,"workshop, tijdsinvester, creatief, aankled,...",2B01F3C0-E1C4-E911-8104-001DD8B72B61,Netwerkevenement,OV-JO-FinFinder3,Offline
72042,FC89CE9B-17CA-E711-80EC-001DD8B72B62,4AFDE787-17CA-E711-80EC-001DD8B72B62,unknown,"contact lidmaatschap, bedrijfsleider",ZOTTEGEM,Aalst,Productie & Diensten,Bedrijf,Bouw,"workshop, tijdsinvester, creatief, aankled,...",39BBF8F7-6F3A-E911-80FC-001DD8B72B61,Project,OV-P-Groep J2 - StartUp-Bryo2019,Offline
72055,FC89CE9B-17CA-E711-80EC-001DD8B72B62,4AFDE787-17CA-E711-80EC-001DD8B72B62,unknown,"contact lidmaatschap, bedrijfsleider",ZOTTEGEM,Aalst,Productie & Diensten,Bedrijf,Bouw,"workshop, tijdsinvester, creatief, aankled,...",52A5376C-6851-E911-80FD-001DD8B72B61,Projectgebonden,OV-P-Plato-Overkoepelende sessie: Veerkracht,Offline
